In [1]:
import pandas as pd

In [19]:
applied_data = pd.read_json("./train_data/Applied_data.json")
aptitude_data_1 = pd.read_json("./train_data/Aptitude.json")
aptitude_data_2 = pd.read_json("./train_data/Aptitude_2.json")
interview_1 = pd.read_json('./train_data/Interview.json')
interview_2 = pd.read_json('./train_data/Interview_Next_Stage_2.json')
oa_data = pd.read_json('./train_data/OA.json')
offer_data_1 = pd.read_json('./train_data/Offer.json')
offer_data_2 = pd.read_json('./train_data/Offer_2.json')
rejected = pd.read_json('./train_data/Rejected.json')

df_jobs = pd.concat([applied_data, aptitude_data_1, aptitude_data_2, interview_1, interview_2, oa_data, offer_data_1, offer_data_2, rejected])
df_jobs['label'] = df_jobs.apply(lambda x: 1,axis=1)

no_job_1 = pd.read_json('./train_data/Non_Job_1.json')
no_job_2 = pd.read_json('./train_data/Non_Job_2.json')
no_job_3 = pd.read_json('./train_data/Non_Job_3.json')

df_no_job = pd.concat([no_job_1,no_job_2,no_job_3])

def remove_n(row):
    text = row['body']
    text = text.replace('\n' ," ")
    return text
df_jobs['body'] = df_jobs.apply(lambda x: remove_n(x),axis=1)
df_no_job['body'] = df_no_job.apply(lambda x: remove_n(x),axis=1)


In [27]:
final_df = pd.concat([df_jobs,df_no_job])
final_df = final_df.sample(frac=1).reset_index(drop=True)

def append_text(row):
    subj = row['subject']
    body = row['body']

    text = "Subject: " + subj + " Body:" + body 
    return text

final_df['text'] = final_df.apply(lambda x: append_text(x),axis=1)
final_df.to_csv("./train_data/job_not_job_data.csv",index=False)

In [28]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="./train_data/job_not_job_data.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [29]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset

# Load a SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")
# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset['train'],
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=4,
    num_iterations=20, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"text": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
trainer.train()

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 5520
  Num epochs = 1
  Total optimization steps = 1380
  Total train batch size = 4


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1380 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
trainer.model._save_pretrained("job_email_classification_model")

## INFERENCE

In [4]:
!zip -r /content/job_email_classification_model.zip /content/job_email_classification_model

'zip' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
from google.colab import files
files.download("/content/job_email_classification_model.zip")

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Download from Hub and run inference
model = SetFitModel.from_pretrained("job_email_classification_model")
# Run inference


In [ ]:
def infer_one_sentence(sentence):
        predtext = [sentence]
        predicted_class = model(predtext)
        return str(predicted_class)

In [ ]:
infer_one_sentence("You have been shortlisted for the interview")

'tensor([1])'

In [ ]:
infer_one_sentence("We appreciate your interest in SAP and the time you’ve invested in applying for the Software Developer Intern (SAP AI) opening. Unfortunately we will not be moving forward with your application, nevertheless we would like to share additional information with you, so please take a minute to watch this video message.")

'tensor([1])'

In [ ]:
infer_one_sentence("please send your NRIC and passport to move on further.")

'tensor([1])'

In [ ]:
infer_one_sentence("""thank you for returning my call earlier! J

As shared, we are happy to offer you an internship opportunity with IHiS for the project: NLP modelling & visualisation on clinical textual data

Kindly refer to the details below for the internship details.

Allowance:                  $1800 per month
Period:                        9 Jan 2023 – 26 May 2023 (credit bearing internship)
Office hours:               830am to 6pm (Monday to Thursday) and 830am to 530pm (Friday), 1h lunch break
Location                      1 Maritime Square #12-01 Harbourfront Centre Singapore 099253""")

'tensor([1])'

In [ ]:
infer_one_sentence("""How are you? Please buy this Vitamins for extra health benefits?""")

'tensor([0])'

In [ ]:
infer_one_sentence("""
Your job alert for machine learning
3 new jobs in Singapore match your preferences.
Singapore University of Technology and Design (SUTD)
Student Intern
Singapore University of Technology and Design (SUTD) · Singapore (On-site)
Synapxe
3 company alumni

Easy Apply
Easy Apply
 Shopee
Analytics Intern - Features & Games, Regional Marketing
Shopee · Singapore (On-site)
gray radar icon
Actively recruiting
 Temasek
Data Science/Analytics Intern, Finance (Financial Management) Jan - Jun/Jul 2024
Temasek · Singapore (Hybrid)
Nanyang Technological University Singapore
1,296 school alumni
See all jobs
""")



'tensor([0])'